In [1]:
import pandas as pd 
import numpy as np 
import openmatrix as omx
from shutil import copytree, rmtree
import yaml
import logging
import os

In [2]:
os.getcwd()

'/home/ubuntu/activitysim/carb'

## Reduce transit fare [0 - 100%]

In [3]:
def create_new_directory(source, destination):
    """ Creates a copy of source (directory) in destination (another directory):
    Parameters
    -----------
    - source: directory path
    - destiantion: directory path 
    
    Returns 
    None 
    """
    logging.info('Create new model directory in {}'.format(destination))
    
    try:
        copytree(source, destination)
    except FileExistsError:
        print ('File {} already exist - Replacing file'.format(destination))
        rmtree(destination)
        copytree(source, destination)


def transit_fare_reduction(skims_path, reduction, modes = None):
    ''' 
    Reduces transit fare by reduction 
    Parameters
    -----------
    - skims: skims file or path 
    - reduction: float. Number between 0 and 1
    - modes: str. Can be "bus", 'rail', 'lightrail' ## To add functionality
    '''
    assert 0<= reduction <=1, 'reduction parameter should be between zero and one'
    logging.info('Modifying transit skims for reduction{:.0f}'.format(reduction*100))
    
    skims = omx.open_file(skims_path,'a')
    skims_modified = omx.open_file(skims_path[:-4] + '_transit_fare_reduction.omx','w')

    matrices = skims.list_matrices()
    for matrix in matrices: 
        try:
            if 'FAR' in(matrix):
                skims_modified[matrix] = np.array(skims[matrix])*(1-reduction)
            else:
                skims_modified[matrix] = np.array(skims[matrix])
        except NodeError:
            pass 
    skims.close()
    skims_modified.close()
    
def modify_settings(settings_path, 
                    sample_size = 0, 
                    num_processes = 60,
                    skims_name = 'skims.omx'):
    
    ''' Modifies the settings.yaml file in the configs files'''
    logging.info('Modifying settings.yaml file')
    a_yaml_file = open(settings_path)
    settings = yaml.load(a_yaml_file, Loader=yaml.FullLoader)
    
    #Modify Settings
    settings['skims_file'] = skims_name
    settings['households_sample_size'] = sample_size
    settings['num_processes'] = num_processes
    
    with open(settings_path, 'w') as outfile:
        yaml.dump(settings, outfile, default_flow_style=False)

In [4]:
def create_scenario(source, destination, reductions, 
                    sample_size = 0, num_processes = 60):
    """
    Create folder to run scenarios 
    
    Parameters
    -----------
    - source: base line scenario path 
    - destination: folder name to store all other files
    - reductions: different scenarios reductions 
    - sample_size: Sample size for each scenario 
    """

    for scenario in reductions:
        #Create new directory
        name = 'reduction_{:.0f}'.format(scenario*100)
        
        #Check if file exist, if it doesn't exist create it 
        if os.path.exists(destination + name):
            replace = False
        else:
            replace = True

            
        if replace:
            #Create new directory for each scenario
            create_new_directory(source, destination + name)

            #Modify transit skims fare
            skims_path = destination + name + '/data/skims.omx'
            transit_fare_reduction(skims_path, scenario)

        #Modify Configs files
        settings_path = destination + name + '/configs/settings.yaml'
        modify_settings(settings_path, 
                        sample_size = sample_size, 
                        skims_name = 'skims_transit_fare_reduction.omx',
                        num_processes = num_processes)


In [5]:
sample_size = 200000
reductions = np.linspace(0, 1, 11) #From 0 to 100 % redcution every 10%
# reductions = [0, 0.1]
source = 'bay_area_base'
destination = 'transit/'
num_processes = 60

create_scenario(source, destination, reductions, 
                sample_size = sample_size, 
                num_processes = num_processes)

In [ ]:
# sample_size = 200000
# reductions = [0, 0.1]
# replace = False
# source = 'bay_area_base'
# destination = 'transit/'


# for scenario in reductions:
#     #Create new directory
#     name = 'reduction_{:.0f}'.format(scenario*100)
    
#     if replace:
#         create_new_directory(source, destination + name)
    
#     #Modify transit skims fare
    
#     skims_path = destination + name + '/data/skims.omx'
#     transit_fare_reduction(skims_path, scenario)
    
#     #Modify Configs files
#     settings_path = destination + name + '/configs/settings.yaml'
#     modify_settings(settings_path, 
#                     sample_size = sample_size, 
#                     skims_name = 'skims_transit_fare_reduction.omx')


In [ ]:
# skims_path = 'transit/reduction_10/data/skims.omx'
# reduction = 0.1

# skims = omx.open_file(skims_path,'a')
# skims_modified = omx.open_file(skims_path[:-4] + '_transit_fare_reduction.omx','w')

# matrices = skims.list_matrices()
# for matrix in matrices: 
#     try:
#         if 'FAR' in(matrix):
#             skims_modified[matrix] = np.array(skims[matrix])*(1-reduction)
#         else:
#             skims_modified[matrix] = np.array(skims[matrix])
#     except NodeError:
#         pass 

## Running models

In [6]:
path = 'transit'

In [9]:
models = os.listdir(path)
to_drop = ['.ipynb_checkpoints']
models = list(set(models) - set(to_drop))

In [11]:
os.chdir(path)
for model in models:
    os.chdir(model)
    %run simulation.py
    os.chdir('../')

INFO - activitysim - Read logging configuration from: configs/logging.yaml
INFO - activitysim - setting households_sample_size: 1000
INFO - activitysim - setting chunk_size: 5000000000
INFO - activitysim - setting multiprocess: True
INFO - activitysim - setting num_processes: 60
INFO - activitysim - setting resume_after: None
DEBUG - activitysim.core.tracing - delete_output_files ignoring output/activitysim.log
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - Setting num_processes = 0 for step mp_households
INFO - activitysim.core.pipeline - open_pipeline
INFO - activitysim.core.random - Set random seed base to 0
DEBUG - activitysim.core.pipeline - open_pipeline - new, empty pipeline
DEBUG - activitysim.core.pipeline - opened pipeline_store
DEBUG - activitysim.core.pipeline - add_checkpoint init timestamp 2021-05-20 01:58:06.482609
DEBUG - activitysim.core.pipeline - open_pipeline complete
INFO - activitysim.core.tracing - Time to execute open_pipeline : 0.02 seconds (0.0 mi

/home/ubuntu/activitysim/activitysim/core/input.py:108: FutureWarning: table_inf option 'column_map' renamed 'rename_columns'Support for 'column_map' will be removed in future versions.
  warnings.warn("table_inf option 'column_map' renamed 'rename_columns'"


INFO - activitysim.core.tracing - Time to execute sub_process mp_setup_skims : 107.111 seconds (1.8 minutes)
INFO - activitysim.core.tracing - Time to execute setup skims : 107.163 seconds (1.8 minutes)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - run_sub_simulations step mp_initialize models resume_after None
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_initialize target=<function mp_run_simulation at 0x7fe28e5bcf70>
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_initialize arg spokesman=True
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_initialize arg queue=<multiprocessing.queues.Queue object at 0x7fe28e2b17f0>
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_initialize arg injectables={'data_dir': 'data', 'configs_dir': ['configs', 'configs/configs'], 'output_dir': 'output'}
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - creat

/home/ubuntu/activitysim/activitysim/abm/models/initialize.py:59: FutureWarning: annotate_tables option 'column_map' renamed 'rename_columns' and movedto settings.yaml. Support for 'column_map' in annotate_tables will be removed in future versions.
  warnings.warn("annotate_tables option 'column_map' renamed 'rename_columns' and moved"


INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_initialize initialize_landuse : 0.657 seconds (0.0 minutes)
mp_initialize WARNING - activitysim.core.assign - numpy: Warning: underflow encountered in exp expression: _decay = _rt_available * exp(_nmDist * dispersion_parameter_walk)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_initialize compute_accessibility : 2.139 seconds (0.0 minutes)


...

INFO - activitysim.core.tracing - Time to execute sub_process mp_households_coalesce : 6.013 seconds (0.1 minutes)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - run_sub_simulations step mp_summarize models resume_after None
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_summarize target=<function mp_run_simulation at 0x7fe28e5bcf70>
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_summarize arg spokesman=True
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_summarize arg queue=<multiprocessing.queues.Queue object at 0x7fe28e42e0d0>
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_summarize arg injectables={'data_dir': 'data', 'configs_dir': ['configs', 'configs/configs'], 'output_dir': 'output'}
DEBUG - activitysim.core.mp_tasks - mp_tasks - MainProcess - create_process mp_summarize arg step_info={'begin': 'generate_beam_plans', 'name': 'mp_summarize',

/home/ubuntu/activitysim/activitysim/core/pipeline.py:289: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['trip_mode', 'ActivityElement', 'ActivityType'], dtype='object')]

  write_df(df, table_name, checkpoint_name)


INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_summarize generate_beam_plans : 64.598 seconds (1.1 minutes)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_summarize write_data_dictionary : 0.049 seconds (0.0 minutes)


/home/ubuntu/anaconda3/envs/asim/lib/python3.9/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->Index(['tour_id', 'household_id', 'person_id', 'participant_num'], dtype='object')]

  pytables.to_hdf(
/home/ubuntu/anaconda3/envs/asim/lib/python3.9/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['trip_mode', 'ActivityElement', 'ActivityType'], dtype='object')]

  pytables.to_hdf(


INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_summarize write_tables : 0.187 seconds (0.0 minutes)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - mp_summarize write_outputs_to_s3 : 0.025 seconds (0.0 minutes)
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - process mp_summarize completed
INFO - activitysim.core.mp_tasks - mp_tasks - MainProcess - Process mp_summarize completed with exitcode 0
INFO - activitysim.core.tracing - Time to execute run_sub_simulations step mp_summarize : 66.202 seconds (1.1 minutes)
INFO - activitysim.core.mem - high water mark rss: 118.79 timestamp: 20/05/2021 02:12:50 label: mp_households_59.trip_purpose_and_destination.completed
INFO - activitysim.core.mem - high water mark used: 9.39 timestamp: 20/05/2021 02:12:35 label: mp_households_55.trip_destination.completed
INFO - activitysim.core.tracing - Time to execute everything : 534.888 seconds (8.9 minutes)


In [13]:
os.chdir('../')

## Analysis of scenario

In [14]:
os.getcwd()

'/home/ubuntu/activitysim/carb'

In [17]:
hdf_store = pd.HDFStore('transit/reduction_10/output/final_output_tables.h5')

In [18]:
hdf_store.keys()

['/checkpoints',
 '/households',
 '/joint_tour_participants',
 '/land_use',
 '/persons',
 '/plans',
 '/tours',
 '/trips']

In [21]:
hdf_store['/trips'].trip_mode.value_counts()

DRIVEALONEFREE    2070
DRIVEALONEPAY     1989
WALK               874
SHARED2FREE        729
SHARED2PAY         719
SHARED3FREE        641
WALK_LOC           601
SHARED3PAY         563
TNC_SINGLE         318
TNC_SHARED         139
WALK_LRF           126
BIKE                98
DRIVE_LOC           86
TAXI                41
WALK_HVY            28
WALK_EXP            27
WALK_COM            17
DRIVE_LRF            5
DRIVE_HVY            5
DRIVE_COM            2
Name: trip_mode, dtype: int64

In [22]:
hdf_store['/trips'].shape

(9078, 14)